

Let's see what kind of things are in EuroVoc, which acts as a thesaurus of places and things that the EU is involved in,
with names and descriptions in all EU languages.

We did this mainly as a potential source for things to find in Named Entity Recognition.

You can find data at:
https://op.europa.eu/en/web/eu-vocabularies/dataset/-/resource?uri=http://publications.europa.eu/resource/dataset/eurovoc

See also [the documentation](https://op.europa.eu/o/opportal-service/euvoc-download-handler?cellarURI=http%3A%2F%2Fpublications.europa.eu%2Fresource%2Fdistribution%2Fdocumentation%2Feurovoc%2Fhandbook%2Fpdf%2Fstddoc%2FEuroVoc-Handbook.pdf&fileName=EuroVoc-Handbook.pdf) linked in the same place

It's there in a number of formats. 
We chose MARCXML for some reason - it's not necearily the easiest to parse.
You may prefer the RDF.

In [ ]:
import wetsuite.helpers.etree

with open('marcxmlout.xml','rb') as f: # open that one big downloaded file
    x = wetsuite.helpers.etree.fromstring( f.read() )
    x = wetsuite.helpers.etree.strip_namespace(x)
    
def df_dict(node):
    ''' helps parse datafield-and-subfied structures intro a dict. 
        In general MARC, fields see repetition that makes this code invalid, but this seems to hold for this particular data. '''
    ret = {}
    ret['tag'] = node.get('tag')
    sfs = {}
    for sf in node.findall('subfield'): # this is not generally valid to do.
        sfs[sf.get('code')] = sf.text
    ret['subfields'] = sfs
    return ret

In [ ]:
concepts = {}

for collection_item in x:#random.sample( list(x), 100):
    concept = {}
    concept['relateds'] = []
    concept['translations'] = {}
    concept['descriptions'] = {}
    for datafield in collection_item.findall('datafield'):
        dd = df_dict(datafield)
        sf = dd['subfields']

        if dd['tag'] == '150':
            #print(sf)
            if sf['9']=='dut':
                concept['naam'] = sf['a']
            concept['translations'][sf['9']] = sf['a']

        elif dd['tag'] == '065':
            try:
                concept['url'] = sf['1']
            except:
                display( '065 fail', sf, wetsuite.helpers.etree.debug_color( datafield ) )

        elif dd['tag'] == '550': # for related to
            try:
                if 'related' in sf.get('i',''):
                    concept['relateds'].append( (sf['i'],sf['1']) )
            except Exception as e:
                display( '550 fail', str(e), sf, wetsuite.helpers.etree.debug_color( datafield ) )

        elif dd['tag'] == '450': # useful? how is it different from 150?

            pass
        elif dd['tag'] == '677': # useful?

            if sf['9']=='dut':
                concept['description'] = sf['a']
            concept['descriptions'][sf['9']] = sf['a']


        elif dd['tag'] == '680': # useful?
            pass
        elif dd['tag'] == '688': # useful?
            pass

        else:
           display( wetsuite.helpers.etree.debug_color( datafield ) )
    
    concepts[ concept['url'] ] = concept
    #display( wetsuite.helpers.etree.debug_color( collection_item ) )
    #display( thing )


len( concepts )

7456

In [ ]:
import random
display( random.choice( list(concepts.values())) )

In [7]:
import pprint
#lang = 'eng'
lang = 'dut'

for url, concept in concepts.items():
    concept_name = concept['translations'].get(lang, '')
    concept_description = concept['descriptions'].get(lang, '')
    #if len(concept['relateds'])>0:
    print()
    print( f'{concept_name:45s}                             {url}' )
    #print( f'{concept_description}' )
    for how, related_url in concept['relateds']:
        print( f"  {how:20s} {concepts[related_url]['translations'][lang]:50s} {related_url}" ) 


enclave                                                                   http://eurovoc.europa.eu/7353
  related to           Ceuta                                              http://eurovoc.europa.eu/6282
  related to           Melilla                                            http://eurovoc.europa.eu/6283
  related to           Kaliningradvraagstuk                               http://eurovoc.europa.eu/8354

rode wijn                                                                 http://eurovoc.europa.eu/4724

voederbiet                                                                http://eurovoc.europa.eu/4857
  related to           veevoer                                            http://eurovoc.europa.eu/4854
  related to           verbouw van voedergewassen                         http://eurovoc.europa.eu/320
  related to           voedergewas                                        http://eurovoc.europa.eu/2412
  related to           knolgewas                              